In [ ]:
import os

import numpy as np
import pandas as pd

import alignparse.targets
import alignparse.minimap2
import alignparse.consensus
from alignparse.constants import CBPALETTE

import dms_variants.utils
import dms_variants.codonvarianttable

from plotnine import *
from io import StringIO

In [ ]:
#Set directory
outdir = 'ap_out' # Put './' if it should be stored at the same place as this notebook.

#Input files
target_file = 'ap_in/file.gb'
parse_specs_file = 'file.yaml'
ccs_folder = './ap_in' # Folder where ccs file(s) are found

#Variables
take_reverse_complement = False
error_rate_floor = 1e-7  # error rates < this set to this
error_cutoff = 1e-4
gen_sequence = 'ATGCATTCTCAAAA...' # From snapgene

#Output file
create_lookup_table = False
lookup_table_filename = 'out.csv'
num_of_indels = 'number_of_indels == 0' # in the final lookup tables

create_variants_lookup_table = False
variants_lookup_table_filename = 'var_out.csv'

#Visualization
display_lookup_table = False
display_variants_lookup_table = True

In [ ]:
os.makedirs(outdir, exist_ok=True)
current_path = os.getcwd()
outdir = os.path.join(current_path,outdir)

targets = alignparse.targets.Targets(
    seqsfile=target_file, #name of the .gb file
    feature_parse_specs=parse_specs_file #name of the .yaml file
)

fastqs = [os.path.join(ccs_folder,d) for d in os.listdir(ccs_folder) if d.endswith('fastq') or d.endswith('fastq.gz')]
run_names = [os.path.basename(os.path.splitext(name)[0]) for name in fastqs]
libraries = run_names.copy()

pacbio_runs = pd.DataFrame( # table with name library and fastq file found 
    {'name': run_names,
    'library': libraries,
    'fastq': fastqs
    })

mapper = alignparse.minimap2.Mapper(alignparse.minimap2.OPTIONS_CODON_DMS)

readstats, aligned, filtered = targets.align_and_parse(
    df=pacbio_runs,
    mapper=mapper,
    outdir=outdir,
    name_col='name',
    queryfile_col='fastq',
    group_cols=['library'],
    overwrite=True,  # overwrite any existing output
)

if take_reverse_complement:
    aligned_df = (aligned[targets.target_names[0]]
                  .assign(barcode=lambda x: x['barcode_sequence'].map(dms_variants.utils.reverse_complement),
                          barcode_error=lambda x: np.clip(1 - x['barcode_accuracy'],error_rate_floor, None),
                          gene_error=lambda x: np.clip(1 - x['gene_accuracy'],error_rate_floor, None),
                          retained=lambda x: ((x['gene_error'] <= error_cutoff) & 
                                              (x['barcode_error'] <= error_cutoff)),
                         )
                 )
else:
    aligned_df = (aligned[targets.target_names[0]]
              .assign(barcode=lambda x: x['barcode_sequence'],
                      barcode_error=lambda x: np.clip(1 - x['barcode_accuracy'],error_rate_floor, None),
                      gene_error=lambda x: np.clip(1 - x['gene_accuracy'],error_rate_floor, None),
                      retained=lambda x: ((x['gene_error'] <= error_cutoff) & 
                                          (x['barcode_error'] <= error_cutoff)),
                     )
             )

aligned_df = alignparse.consensus.add_mut_info_cols(
    aligned_df,
    mutation_col='gene_mutations',
    n_indel_col='n_indels'
).assign(has_indel=lambda x: x['n_indels'] > 0)

#aligned_df = aligned_df[['library', 'name', 'query_name', 'barcode', 'gene_mutations','barcode_accuracy', 'gene_accuracy', 'barcode_error', 'gene_error', 'retained', 'n_indels', 'has_indel']]

consensus, dropped = alignparse.consensus.simple_mutconsensus(
    aligned_df.query('retained'),
    group_cols=('library', 'barcode'),
    mutation_col='gene_mutations',
)

consensus = alignparse.consensus.add_mut_info_cols(
    consensus,
    mutation_col='gene_mutations',
    sub_str_col='substitutions',
    n_indel_col='number_of_indels',
    overwrite_cols=True
)

lookup_table = consensus.query(num_of_indels)

if create_lookup_table:
    lookup_table.to_csv(os.path.join(outdir,lookup_table_filename))

if display_lookup_table:
    display(lookup_table)    

variants = dms_variants.codonvarianttable.CodonVariantTable(
    barcode_variant_file = StringIO(lookup_table.to_csv()),
    geneseq = gen_sequence,
)

variants_lookup_table = variants.barcode_variant_df

if create_variants_lookup_table:
    variants_lookup_table.to_csv(os.path.join(outdir,variants_lookup_table_filename))
    
if display_variants_lookup_table:
    display(variants_lookup_table)